
# Tutorial 02: Simulating Growth - Deposition MD and Long-Timescale Ordering (aKMC)

**Goal**: Demonstrate the system's ability to handle complex, multi-scale physics using the potential trained in Tutorial 01.

This tutorial guides you through:
1.  **Setup**: Loading the potential.
2.  **Phase A (Deposition)**: LAMMPS MD simulation of Fe/Pt deposition.
3.  **Phase B (Ordering)**: EON aKMC simulation of L10 ordering.
4.  **Analysis**: Visualizing results.


In [ ]:

import os
import sys
from pathlib import Path
import shutil

# Add src to path
if Path("../src").exists():
    sys.path.append(str(Path("../src").resolve()))

from mlip_autopipec.core.orchestrator import Orchestrator
from mlip_autopipec.domain_models.config import (
    GlobalConfig,
    DynamicsConfig,
    EONConfig,
    GeneratorConfig,
    OrchestratorConfig,
    OracleConfig,
    TrainerConfig,
    ValidatorConfig
)
from mlip_autopipec.domain_models.enums import DynamicsType, ExecutionMode
from mlip_autopipec.dynamics.interface import MockDynamics
from mlip_autopipec.domain_models.potential import Potential
from mlip_autopipec.domain_models.structure import Structure


In [ ]:

# Detect Mode
IS_CI_MODE = os.environ.get("CI", "false").lower() == "true"
WORK_DIR = Path("outputs/02_Deposition")
WORK_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:

# Step 1: Load Potential
# In a real scenario, we would load 'outputs/01_MgO_FePt/active_potential.yace'
# For this tutorial, we will use a Mock potential in CI mode.

potential_path = Path("outputs/01_MgO_FePt/active_potential.yace")

if IS_CI_MODE:
    # Create a dummy potential file if it doesn't exist
    if not potential_path.exists():
        potential_path.parent.mkdir(parents=True, exist_ok=True)
        potential_path.write_text("MOCK_POTENTIAL_CONTENT")

print(f"Using potential: {potential_path}")
active_potential = Potential(path=potential_path, format="yace")


In [ ]:

# Step 2: Phase A - Deposition MD (LAMMPS)

if IS_CI_MODE:
    # Use Mock Dynamics
    dyn_config = DynamicsConfig(
        type=DynamicsType.MOCK,
        steps=100,
        mock_frames=5
    )
    # We need a dummy structure to start
    # Create a simple structure (e.g. 2 atoms)
    from ase import Atoms
    atoms = Atoms('CO', positions=[[0, 0, 0], [0, 0, 1.2]], cell=[10, 10, 10], pbc=True)
    initial_structure = Structure(atoms=atoms, provenance="manual_setup")
    
    # Instantiate MockDynamics directly for demonstration if not going through Orchestrator
    # Or use Orchestrator logic. Here we use the component directly to show flexibility.
    
    dynamics = MockDynamics(dyn_config)
    print("Running Mock Deposition MD...")
    trajectory = list(dynamics.simulate(active_potential, initial_structure))
    print(f"Deposition complete. Generated {len(trajectory)} frames.")
    final_structure = trajectory[-1]
    
else:
    # Real LAMMPS Deposition
    print("Running Real LAMMPS Deposition...")
    from mlip_autopipec.dynamics.lammps_driver import LAMMPSDriver
    
    dyn_config = DynamicsConfig(
        type=DynamicsType.LAMMPS,
        steps=5000,
        temperature=600.0,
        lammps_command="lmp",
        halt_on_uncertainty=True
    )
    
    try:
        driver = LAMMPSDriver(WORK_DIR, dyn_config)
        # Note: In a real deposition simulation, we would insert atoms iteratively.
        # Here we run MD on the initial structure for demonstration.
        trajectory = list(driver.simulate(active_potential, initial_structure))
        final_structure = trajectory[-1]
        print(f"MD complete. Final energy: {final_structure.energy}")
    except Exception as e:
        print(f"Real execution failed (likely missing 'lmp'): {e}")
        print("Falling back to initial structure.")
        final_structure = initial_structure


In [ ]:

# Step 3: Phase B - Ordering aKMC (EON)

if IS_CI_MODE:
    # Mock EON
    print("Running Mock aKMC Ordering...")
    ordered_structure = final_structure.model_copy(deep=True)
    # Simulate ordering effect
    if hasattr(ordered_structure.atoms, "set_chemical_symbols"):
         # Swap some atoms to simulate ordering if possible, or just keep as is
         pass
    print("aKMC complete.")
    
else:
    # Real EON execution
    print("Running Real EON aKMC...")
    from mlip_autopipec.dynamics.eon_driver import EONDriver
    
    eon_config = EONConfig(
        temperature=500.0,
        client_path="eonclient"
    )
    dyn_config = DynamicsConfig(
        type=DynamicsType.EON,
        eon=eon_config
    )
    
    try:
        driver = EONDriver(WORK_DIR, dyn_config)
        # EONDriver simulate yields structures (saddle points, minima)
        trajectory = list(driver.simulate(active_potential, final_structure))
        if trajectory:
            ordered_structure = trajectory[-1]
            print(f"aKMC complete. Found {len(trajectory)} states.")
        else:
             print("aKMC found no new states.")
             ordered_structure = final_structure
    except Exception as e:
        print(f"Real execution failed (likely missing 'eonclient'): {e}")
        print("Falling back to MD structure.")
        ordered_structure = final_structure


In [ ]:

# Step 4: Analysis & Visualization
# Visualize the final structure

try:
    from ase.visualize import view
    # view(ordered_structure.to_ase()) # Interactive view (not visible in CI)
    print("Structure ready for visualization.")
    print(ordered_structure.to_ase())
except ImportError:
    print("ASE GUI not available.")
